<a href="https://colab.research.google.com/github/Ar1stok/NLP/blob/main/ner_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Распознавание именованных сущностей с использованием BERT

**Описание:**

Целью данной лабораторной работы является применение модели BERT для распознавания именованных сущностей (NER) на текстах из астрофизики. Вы обучите модель на специализированном датасете и оцените её качество, используя метрику F1 для сущностей.

## 1. Настройка среды

- Установите необходимые библиотеки: `transformers`, `datasets`, `seqeval`, `evaluate`, `spacy`, `tensorboard`.
- Инструменты: `pip` для установки библиотек.

In [1]:
!pip install -q datasets transformers evaluate seqeval

In [2]:
from datasets import load_dataset, Dataset
import evaluate
import numpy as np
from sklearn.metrics import classification_report
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer
from transformers import pipeline

In [3]:
TOKEN_MODEL_NAME = 'google-bert/bert-base-cased'
DATASET_NAME = 'adsabs/WIESP2022-NER'
MODEL_NAME = 'FacebookAI/xlm-roberta-base'
BATCH_SIZE = 4
EPOCH = 10
LR = 5e-5

## 2. Загрузка и предварительная обработка данных

- Датасет: Используйте набор данных WIESP2022-NER из библиотеки `datasets`.
- Загрузите данные и изучите структуру набора: обратите внимание на разметку сущностей и формат данных.
- Разделите данные на обучающую, валидационную и тестовую выборки.
- Инструменты: библиотека `datasets`.

In [4]:
data = load_dataset(DATASET_NAME)
data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['bibcode', 'label_studio_id', 'ner_ids', 'ner_tags', 'section', 'tokens', 'unique_id'],
        num_rows: 1753
    })
    validation: Dataset({
        features: ['bibcode', 'label_studio_id', 'ner_ids', 'ner_tags', 'section', 'tokens', 'unique_id'],
        num_rows: 1366
    })
    test: Dataset({
        features: ['bibcode', 'label_studio_id', 'ner_ids', 'ner_tags', 'section', 'tokens', 'unique_id'],
        num_rows: 2505
    })
})

In [5]:
labels = set()
for example in data["train"]:
    labels.update(example["ner_tags"])
labels = sorted(labels)
num_labels = len(labels)

In [6]:
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for label, i in label2id.items()}

id2label

{0: 'B-Archive',
 1: 'B-CelestialObject',
 2: 'B-CelestialObjectRegion',
 3: 'B-CelestialRegion',
 4: 'B-Citation',
 5: 'B-Collaboration',
 6: 'B-ComputingFacility',
 7: 'B-Database',
 8: 'B-Dataset',
 9: 'B-EntityOfFutureInterest',
 10: 'B-Event',
 11: 'B-Fellowship',
 12: 'B-Formula',
 13: 'B-Grant',
 14: 'B-Identifier',
 15: 'B-Instrument',
 16: 'B-Location',
 17: 'B-Mission',
 18: 'B-Model',
 19: 'B-ObservationalTechniques',
 20: 'B-Observatory',
 21: 'B-Organization',
 22: 'B-Person',
 23: 'B-Proposal',
 24: 'B-Software',
 25: 'B-Survey',
 26: 'B-Tag',
 27: 'B-Telescope',
 28: 'B-TextGarbage',
 29: 'B-URL',
 30: 'B-Wavelength',
 31: 'I-Archive',
 32: 'I-CelestialObject',
 33: 'I-CelestialObjectRegion',
 34: 'I-CelestialRegion',
 35: 'I-Citation',
 36: 'I-Collaboration',
 37: 'I-ComputingFacility',
 38: 'I-Database',
 39: 'I-Dataset',
 40: 'I-EntityOfFutureInterest',
 41: 'I-Event',
 42: 'I-Fellowship',
 43: 'I-Formula',
 44: 'I-Grant',
 45: 'I-Identifier',
 46: 'I-Instrument',
 47

## 3. Подготовка токенов для модели

- Загрузите токенайзер для модели BERT с помощью `transformers` (например, bert-base-cased).
- Примените токенизацию к текстам, сохраняя разметку сущностей.
- Учтите, что некоторые сущности могут быть разбиты на несколько токенов.
- Инструменты: `AutoTokenizer` из `transformers`.

In [7]:
tokenizer = AutoTokenizer.from_pretrained(TOKEN_MODEL_NAME)

In [8]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], max_length=512, padding='max_length', truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_ids"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_idx])
        # Also labels are padded to max_length
        label_ids += [-100] * (512 - len(label_ids))
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = data.map(tokenize_and_align_labels, batched=True)

In [9]:
test_dataset = tokenized_datasets["test"]
tokenized_datasets = tokenized_datasets.remove_columns(["tokens","ner_tags"])

In [10]:
train_dataset = tokenized_datasets["train"]
valid_dataset = tokenized_datasets["validation"]

In [11]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [12]:
seqeval = evaluate.load("seqeval")

def compute_metrics(eval_preds, label2id, id2label):
  pred_logits, labels = eval_preds

  pred_logits = np.argmax(pred_logits, axis=2)

  predictions = [
      [id2label[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(pred_logits, labels)
  ]

  references = [
      [id2label[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(pred_logits, labels)
  ]

  results = seqeval.compute(predictions=predictions, references=references)
  return {
      "precision": results["overall_precision"],
      "recall": results["overall_recall"],
      "f1": results["overall_f1"],
      "accuracy": results["overall_accuracy"],
  }

## 4. Создание модели и настройка обучения

- Модель: Загрузите предобученную модель BERT для классификации токенов.
- Настройте модель для задачи NER, добавив нужное количество меток для классификации токенов.
- Задайте параметры обучения, такие как размер батча, количество эпох и метод оптимизации.
- Инструменты: `AutoModelForTokenClassification`, `TrainingArguments`, `Trainer` из `transformers`.

In [13]:
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, num_labels=num_labels)

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCH,
    weight_decay=0.01,
    learning_rate=LR,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

## 5. Обучение модели

- Используйте `Trainer` для обучения модели на обучающем наборе данных.
- Мониторьте метрики обучения с помощью `tensorboard`.
- Инструменты: `Trainer`, `tensorboard`.

In [15]:
trainer = Trainer(
   model=model,
   args=args,
   train_dataset=train_dataset,
   eval_dataset=valid_dataset,
   data_collator=data_collator,
   processing_class=tokenizer,
   compute_metrics=lambda x: compute_metrics(x, label2id, id2label)
)

In [16]:
results = trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: p-krylov (p-krylov-nsu). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.748816,0.420740,0.464482,0.441530,0.822338
2,1.087100,0.483367,0.623738,0.579202,0.600645,0.881780
3,0.506600,0.398282,0.648589,0.643773,0.646172,0.899682
4,0.345700,0.374197,0.664767,0.693681,0.678916,0.907075
5,0.250200,0.370338,0.667785,0.707407,0.687025,0.908663
6,0.183000,0.341300,0.686846,0.736922,0.711003,0.915034
7,0.145400,0.353021,0.701320,0.737200,0.718813,0.916987
8,0.115000,0.353000,0.711476,0.744563,0.727644,0.919832
9,0.115000,0.357885,0.710048,0.750345,0.729640,0.919828
10,0.091800,0.364543,0.715965,0.749143,0.732178,0.920648


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

## 6. Оценка модели

- Оцените модель на тестовой выборке, используя метрику F1 для сущностей.
- Сравните F1 по сущностям и по токенам с помощью библиотеки `seqeval`.
- Инструменты: `evaluate`, `seqeval`.

In [17]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.36454305052757263,
 'eval_precision': 0.7159645983459882,
 'eval_recall': 0.749142893288633,
 'eval_f1': 0.7321780739769896,
 'eval_accuracy': 0.9206476369728265,
 'eval_runtime': 56.2204,
 'eval_samples_per_second': 24.297,
 'eval_steps_per_second': 6.083,
 'epoch': 10.0}

## 7. Визуализация результатов

- Визуализируйте предсказанные сущности в тексте с помощью `spacy` и его модуля `displacy`.
- Проанализируйте ошибки модели и случаи, где сущности распознаны неправильно.
- Инструменты: `spacy`, `displacy`.